<a href="https://colab.research.google.com/github/luckyswaminathan/MusicVibe/blob/main/tweetemotionanalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Import relevant modules
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import nltk as nl
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
nl.download('stopwords')
nl.download('punkt')
from nltk.corpus import stopwords

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:

#imports csv
url = 'https://raw.githubusercontent.com/luckyswaminathan/MusicVibe/main/tweet_emotions.csv'

msc_df = pd.read_csv(url)
msc_df.drop_duplicates(keep='first')
msc_df.drop(columns="tweet_id", inplace = True)
msc_df = msc_df.reindex(np.random.permutation(msc_df.index))

 #defining and removing stop words
stopWords = set(stopwords.words('english'))
stopWords.add(".")
stopWords.add(":")
stopWords.add("they're")



## msc_df['content'] = (msc_df['content']).apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopWords]))
 #tokenizing each sentence in msc_df 

tknzr = TweetTokenizer()

msc_df['content'] = msc_df['content'].apply(lambda x: tknzr.tokenize(x.lower()))
##msc_df['content'] = (msc_df['content']).apply(word_tokenize)


msc_df['content'] = msc_df['content'].apply(lambda x: [word for word in x if word not in (stopWords)]) 
print(msc_df.head())
# # splitting the input/output parts




rows = msc_df.shape[0]
maxTrain =int(rows*0.8)

words_train = msc_df[0:maxTrain]['content']
words_test = msc_df[maxTrain+1:-1]['content']



feelings_train = msc_df[0:maxTrain]['sentiment']
feelings_test = msc_df[maxTrain+1:-1]['sentiment']


# just a print to show the 13 different feelings in this data set
print(np.unique(feelings_train))


       sentiment                                            content
15094  happiness  [can't, spell, #melo, without, e, ..., <3, fav...
35986       love          [salon, mom, happy, mothers, day, mom, !]
24664        fun          [@hollaatk, hey, good, weekend, buddy, ?]
24792      empty  [@lenasvenson, thought, might, interested, @tw...
23637     relief  [@gothtart, congratulations, ,, two, well, sui...
['anger' 'boredom' 'empty' 'enthusiasm' 'fun' 'happiness' 'hate' 'love' 'neutral' 'relief' 'sadness' 'surprise' 'worry']


In [ ]:
#defining model

def createmodel(my_inputs, my_features):
  

  


SyntaxError: ignored